In [1]:
import torch
import os
from PIL import Image
from facenet_pytorch import InceptionResnetV1
import numpy as np
from sklearn.preprocessing import normalize
from torchvision import transforms  # Importing the necessary transforms

dataset_path = "/Users/rajataggarwal/Desktop/dataset"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define a transform pipeline (only resize and convert to tensor)
transform = transforms.Compose([
    transforms.Resize((160, 160)),  # Resizing using transforms
    transforms.ToTensor(),  # Converting to tensor and scaling the pixel values between [0, 1]
])

embeddings_list = []
name_list = []

for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    if not os.path.isdir(person_folder):
        continue

    for image_name in os.listdir(person_folder):
        if not image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        image_path = os.path.join(person_folder, image_name)
        try:
            img = Image.open(image_path).convert('RGB')
            img_tensor = transform(img).unsqueeze(0).to(device)  # Applying the transform
            with torch.no_grad():
                embedding = model(img_tensor).squeeze().cpu().numpy()
                embeddings_list.append(embedding)
                name_list.append(person_name)
        except Exception as e:
            print(f"Failed to process {image_path}: {e}")

# Normalize embeddings (L2 normalization)
embeddings_array = np.array(embeddings_list)
normalized_embeddings = normalize(embeddings_array, axis=1)
normalized_embeddings = normalized_embeddings.astype(np.float32)
# Save normalized embeddings and names
np.savez_compressed(f"{dataset_path}/embeddings.npz", embeddings=normalized_embeddings, names=name_list)

print("Embeddings extraction and normalization complete!")


Embeddings extraction and normalization complete!
